In [ ]:
# https://fontsup.com/es/font/matrix-regular-small-caps.html
_ = !wget -O card_name.ttf https://www.dropbox.com/s/tfwt68u14ncm2zo/MatrixRegularSmallCaps.ttf?dl=1
# https://fontsup.com/es/font/matrix-bold-small-caps.html
_ = !wget -O card_number.ttf https://www.dropbox.com/s/gil5eef4x900nao/MatrixBoldSmallCaps_29827.ttf?dl=1
# https://fontsup.com/es/font/matrix-book.html
_ = !wget -O card_effect.ttf https://www.dropbox.com/s/0psxpekkklsjuw8/Matrix-Book.ttf?dl=1

In [ ]:
import json
from pathlib import Path
import csv
from collections import Counter, defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

import calendar

from matplotlib import font_manager as fm, rcParams
from matplotlib.offsetbox import AnchoredText
from datetime import date
import re

In [ ]:
other_names = {
    "Nurse Reficule the Fallen One": "Darklord Nurse Reficule",
    "Kinetic Soldier": "Cipher Soldier",
    "Call Of The Haunted":"Call of the Haunted",
    "Polymerization (Fusion)": "Polymerization",
    "M-Warrior 1": 'M-Warrior #1',
    "M-Warrior 2": 'M-Warrior #2',
    "Tri-Gate Wizard": "Tri-gate Wizard",
    "Oafdragon Magician\u200e":"Oafdragon Magician",
    "Sea Archiver\u200e\u200e": "Sea Archiver",
    "Utopic Onomatopoeia": "Utopic Onomatopeia",
    "Beta The Magnet Warrior": "Beta the Magnet Warrior"
}
other_names = {k.lower(): v.lower() for k,v in other_names.items()}

month_to_id = {v: k for k,v in enumerate(calendar.month_abbr)}

In [ ]:
card_by_name = dict()
card_by_id = dict()
with open("/kaggle/input/yugioh-cards/cards.csv") as fp:
    reader = csv.DictReader(fp)
    for card in reader:
        card_by_name[card["name"].lower()] = card
        card_by_id[card["id"]] = card

        
deck_count = 0
main_deck_card_counter = Counter()
for linea_deck in Path("/kaggle/input/yugiohtopdecks/decks.ndjson").open():
    deck = json.loads(linea_deck)
    month, _, year = deck["played_in"][deck["played_in"].find("(")+1:-1].partition(" ")
    #print(date(int(year), month_to_id[month]))
    if "deck" in deck and "main" in deck["deck"] and len(deck["deck"]["main"]):
        main_deck = {card_by_name[other_names.get(card_name.lower(), card_name.lower().lower())]["id"] for card_name in deck["deck"]["main"]}
        main_deck_card_counter.update(main_deck)
        deck_count += 1

In [ ]:
other_names["nurse reficule the fallen one"]

In [ ]:
mas_comunes = main_deck_card_counter.most_common(10)
card_ids, counts = list( zip(*mas_comunes) )
card_ids, counts = list(card_ids), list(counts)

En esta ocasión les quiero hablar de cómo mejorar sus gráficos de barras con colores y fuentes personalizadas... aunque me gustaría que esta fuera una lección sobre cómo es que podemos sacar más provecho de Matplotlib y herramientas (como Seaborn) que se construyen al rededor de esta poderosa librería.  

## Los datos  
Ultimamente he estado obsesionado con Yu-Gi-Oh! un juego de cartas en donde cada jugador elije cartas para su baraja, y encontré un dataset que contiene muchos, muchos decks... en fin, no es necesario que sepas nada sobre el juego para seguir este tutorial. Lo importante es que me hice de un dataset de barajas, y quise averiguar cuáles eran las cartas más usadas del juego... en fin, supongamos que tenemos un par de listas `card_ids` y `counts` con los identificadores de las cartas más comunes y la cantidad en la que aparecen en todas las barajas que encontré.

In [ ]:
print(card_ids)
print(counts)

Que indica que el id `83764719` aparece `4712` veces, el id `14558127` lo hace `4479` veces y así sucesivamente...

Además, tengo un diccionario que me permite obtener más información sobre determinada carta a partir de su identificador:  


In [ ]:
card_by_id['83764719']

Tenemos cosas como el nombre y el tipo de carta, esos atributos nos serán de utilidad más adelante.

## Ahora sí, a graficar  

Tenemos ids de cartas y la cantidad de veces que aparecen, suena a un trabajo para... ¡una gráfica de barras! así es como la harías usando *matplotlib*: 

In [ ]:
plt.figure()
plt.bar(card_ids, counts)

### Tamaño de la gráfica  
Además de las críticas obvias (gráfica sin título, no hay etiquetas en los ejes...) la gráfica es horriblemente pequeña! para arreglar esto, podemos jugar con los argumentos `dpi` y `figsize`:    


In [ ]:
figura = plt.figure(figsize=(10,5), dpi=100)
ax = figura.gca()
ax.bar(x=card_ids,height=counts)

Un poquito mejor, ¿cierto? los argumentos que usamos nos ayudan a controlar dos cosas:  
 - `figsize` controla el tamaño de la gráfica en pulgadas; el primer valor de la tupla es el valor horizontal y el segundo es vertical
 - `dpi` controla los *puntos por pulgada* en nuestra gráfica, va de la mano con `figsize`. El usar este argumento nos facilitará en demasía el controlar el tamaño del texto en nuestra gráfica.  
 
 
### La API orientada a objetos  
Para permitirnos aún más personalización, tendremos que aventurarnos a usar la no tan conocida API orientada a objetos de *matplotlib*, pero como verás no es nada del otro mundo:  

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)
ax = fig.gca()
ax.bar(card_ids, counts)

La gran diferencia es que no tenemos ya más referencias a `plt` más allá de la primera que nos ayuda a crear una figura. La novedad, también radica en que usamos el método `gca` (que proviene de *get current axes*) para obtener el *axes* que reside dentro de nuestra figura. Una vez hecho esto, ya podemos empezar a personalizar la gráfica:  

In [ ]:
fig = plt.figure(figsize=(10,5), dpi=100)
ax = fig.gca()
ax.bar(card_ids, counts)

ax.set_xlabel("Card id")
ax.set_ylabel("Occurrences")
ax.set_title(f"Most used cards across {deck_count} Yu-Gi-Oh! decks")

### ¡Entra *Seaborn*!  
Muchas veces, si bien podemos lograr todos los resultados que queramos usando solemente matplotlib, hay ocasiones en las que podemos delegar este trabajo a otras librerías como *Seaborn*. Digamos que queremos mejorar aún más nuestra gráfica usando barras horizontales y los nombres de las cartas:  

In [ ]:
card_names = [card_by_id[card_id]["name"] for card_id in card_ids]
fig = plt.figure(dpi=100, figsize=(10,5))
ax = fig.gca()
sns.barplot(x=counts, y=card_names, ax=ax, orient="h")
ax.set_ylabel("Card name")
ax.set_xlabel("Occurrences")
ax.set_title(f"Most used cards across {deck_count} Yu-Gi-Oh! decks")

La novedad es que estamos usando *seaborn* (lo renombramos como `sns` en nuestro script) y, algo muy importante, le etamos pasando el argumento `ax` para que haga uso del axis que nosotros queremos.  

Se ve un poco mejor, pero la podemos hacer aún más atractiva.  

### Personalización máxima
Ahora podemos comenzar a crear nuestra gráfica:  

In [ ]:
def find_colors(card_type):
    if "Monster" in card_type:
        return "#FF8B53", "black"
    if "Trap" in card_type:
        return "#BC5A84", "white"
    if "Spell" in card_type:
        return "#1D9E74", "white"

In [ ]:
# fm es matplotlib.font_manager
card_name_prop = fm.FontProperties(fname="card_name.ttf", size=30)
card_effect_prop = fm.FontProperties(fname="card_effect.ttf", size=40)
card_effect_prop_sm = fm.FontProperties(fname="card_effect.ttf", size=20)
card_number_prop = fm.FontProperties(fname="card_number.ttf", size=30)

fig = plt.figure(figsize=(15,7), dpi=300)
ax = fig.gca()
sns.barplot(x=counts, y=card_names, ax=ax, orient="h")

#ax.axes.get_yaxis().set_visible(False)
ax.axes.get_yaxis().set_ticks([])

ax.set_ylabel("Card", fontproperties=card_number_prop)
ax.set_xlabel("Occurrences", fontproperties=card_number_prop)
ax.set_title(f"Most used cards across {deck_count} Yu-Gi-Oh! decks", fontproperties=card_effect_prop)

for label in ax.get_xticklabels() :
    label.set_fontproperties(card_number_prop)
    
for card_id, rect in zip(card_ids, ax.patches):
    x, y = rect.xy
    card = card_by_id[card_id]
    rect_color, font_color = find_colors(card["type"])
    rect.set_facecolor(rect_color)
    ax.text(30 , y + 0.6, card["name"], color=font_color, fontproperties=card_name_prop)

text = AnchoredText("Data from all decks available at yugiohtopdecks.com", loc=4,
                    prop={'size': 10,}, frameon=True)
ax.add_artist(text)
pass

Como siempre, comentarios y dudas son bienvenidos en mi cuenta de Twitter [@io_exception](https://twitter.com/io_exception). Esta vez todo el código para reproducir lo que hicimos aquí está en este [notebook de Kaggle](https://www.kaggle.com/ioexception/most-used-cards-across-all-decks) (si no sabes qué es Kaggle, puedes ver [mi video sobre el tema](https://www.youtube.com/watch?v=5yF-VeivtgU)).